由於lingua.com採用SPA，因此只使用bs4只會抓到未經瀏覽器解析前的HTML，因此我們必須利用selenium預先解析出正確的html，在使用bs4進行抓取資料。

In [56]:
from selenium import webdriver
from bs4 import BeautifulSoup

# 创建一个Chrome浏览器实例
driver = webdriver.Chrome()

# 打开目标页面
driver.get("https://lingua.com/english/reading/london/")

# 等待页面完全加载
driver.implicitly_wait(10)

# 获取解析后的HTML内容
html = driver.execute_script("return document.documentElement.outerHTML")

# 使用BeautifulSoup解析HTML
soup = BeautifulSoup(html, 'html.parser')

# 提取data-filename属性
filename = soup.select_one('.speaker-box-info')['data-filename']

# 打印结果
print(filename)

# 关闭浏览器
driver.quit()

KeyError: 'data-filename'

如果使用Selenium仍然无法获取到想要的data-filename属性，可能是因为该属性是在网页中使用JavaScript生成的，而不是在原始HTML代码中直接存在的。
这种情况下，你需要通过模拟用户在浏览器中的操作来获取JavaScript生成的内容。

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# 创建一个Chrome浏览器实例
driver = webdriver.Chrome()

# 打开目标页面
driver.get("https://lingua.com/english/reading/london/")

# 等待元素出现
wait = WebDriverWait(driver, 10)
element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "wrapper-lingua-speakers")))


# 等待页面完全加载
# driver.implicitly_wait(10)

# 模拟用户点击
driver.execute_script("document.querySelector('.speaker-box-info').click();")

# # 等待数据加载
# driver.implicitly_wait(10)

# 获取解析后的HTML内容
html = driver.execute_script("return document.documentElement.outerHTML")

# 使用BeautifulSoup解析HTML
soup = BeautifulSoup(html, 'html.parser')

# 提取data-filename属性
filename = soup.select_one('.speaker-box-info')['data-filename']

# 打印结果
print(filename)

# 关闭浏览器
driver.quit()

KeyError: 'data-filename'


發現了，需要login才有data-filename :
https://lingua.com/login/google/

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 初始化浏览器
driver = webdriver.Chrome()

# 打开登录页面
driver.get("https://lingua.com/login/google/")

# 等待页面加载完成
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.NAME, "identifier")))

# 输入 Google 账号和密码
driver.find_element(By.NAME, "identifier").send_keys("your_account")
driver.find_element(By.NAME, "identifier").send_keys(Keys.RETURN)
wait.until(EC.presence_of_element_located((By.NAME, "password")))
driver.find_element(By.NAME, "password").send_keys("your_password")
driver.find_element(By.NAME, "password").send_keys(Keys.RETURN)

# 等待登录成功，通过检查导航栏中的用户名来判断
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".nav-user-avatar")))

# 跳转到阅读页面
driver.get("https://lingua.com/english/reading/london/")

# 等待元素出现
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "wrapper-lingua-speakers")))

# 获取页面源码
html = driver.page_source

# 解析页面源码
soup = BeautifulSoup(html, "html.parser")

# 获取所有的 data-filename 属性
for tag in soup.select(".wrapper-lingua-speakers"):
    print(tag.get("data-filename"))

# 关闭浏览器
driver.quit()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=111.0.5563.111)
Stacktrace:
Backtrace:
	(No symbol) [0x0022DCE3]
	(No symbol) [0x001C39D1]
	(No symbol) [0x000D4DA8]
	(No symbol) [0x000BD0D3]
	(No symbol) [0x0011EA8B]
	(No symbol) [0x0012D093]
	(No symbol) [0x0011ACC6]
	(No symbol) [0x000F6F68]
	(No symbol) [0x000F80CD]
	GetHandleVerifier [0x004A3832+2506274]
	GetHandleVerifier [0x004D9794+2727300]
	GetHandleVerifier [0x004DE36C+2746716]
	GetHandleVerifier [0x002D6690+617600]
	(No symbol) [0x001CC712]
	(No symbol) [0x001D1FF8]
	(No symbol) [0x001D20DB]
	(No symbol) [0x001DC63B]
	BaseThreadInitThunk [0x75D4FEF9+25]
	RtlGetAppContainerNamedObjectPath [0x77DE7BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77DE7B8E+238]
	(No symbol) [0x00000000]


chrome阻擋了從selenium的登入，改為使用cookie登入。

需要注意的是，cookie_str是从Set-Cookie响应头中获取的，需要将其解析成一个字典对象，然后使用add_cookie()方法将其添加到Selenium的Cookie中。解析Cookie字符串时，需要注意不同的Cookie属性，如expires、path、domain、secure和httpOnly等。如果Cookie没有设置这些属性，可以设置为None或False。

先打開打开目标页面，還是先加入cookie?
一般情况下，建议先打开目标页面，再加入Cookie。这样可以保证Cookie是在目标页面中设置的，避免一些可能的问题。

In [59]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests

# 初始化浏览器
driver = webdriver.Chrome()

# 打开登录页面
# driver.get("https://lingua.com/login/google/")



# 等待页面加载完成
# wait = WebDriverWait(driver, 10)
# wait.until(EC.presence_of_element_located((By.NAME, "identifier")))

# # 输入 Google 账号和密码
# driver.find_element(By.NAME, "identifier").send_keys("your_account")
# driver.find_element(By.NAME, "identifier").send_keys(Keys.RETURN)
# wait.until(EC.presence_of_element_located((By.NAME, "password")))
# driver.find_element(By.NAME, "password").send_keys("your_password")
# driver.find_element(By.NAME, "password").send_keys(Keys.RETURN)

# # 获取cookie
# cookies = driver.get_cookies()

# # 等待登录成功，通过检查导航栏中的用户名来判断
# wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".nav-user-avatar")))

# 将Cookie字符串解析成字典
cookie_str = 'lingua_version=EN%3B1679931692%3Breading; expires=Wed, 26-Apr-2023 15:41:32 GMT; Max-Age=2592000; path=/; domain=lingua.com; secure; httponly'
cookie_parts = cookie_str.split(';')
cookie_dict = {}
for part in cookie_parts[:-2]:
    key, value = part.strip().split('=', 1)
    cookie_dict[key] = value

# 将Cookie添加到Selenium的Cookie中
driver.add_cookie({
    'name': 'lingua_version',
    'value': cookie_dict.get('lingua_version',None),
    'expires': cookie_dict.get('expires', None),
    'Max-Age': cookie_dict.get('Max-Age', None),
    'path': cookie_dict.get('path', None),
    'domain': 'lingua.com',#'.'+cookie_dict.get('domain', None),
    'secure': cookie_dict.get('secure', None),
    'httpOnly': cookie_dict.get('httpOnly', None)
})

for cookie_name, cookie_value in cookie_dict.items():
    driver.add_cookie({'name': cookie_name, 'value': cookie_value})

# 跳转到阅读页面
driver.get("https://lingua.com/english/reading/london/")

# 等待元素出现
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "wrapper-lingua-speakers")))

# 获取页面源码
html = driver.page_source

# 解析页面源码
soup = BeautifulSoup(html, "html.parser")

# 获取所有的 data-filename 属性
# data_filenames = [elem['data-filename'] for elem in soup.select('[data-filename]')]

# 找到 class 属性为 "speaker-box"，并且包含 "American English" 文本的 div 元素
american_div = soup.select_one('.speaker-box:contains("American English")')

# 获取其 data-filename 属性
data_filenames = american_div['data-filename']
url_prefix = "https://lingua.com/"
filepath_prefix = "D:/lingua/"
for f in data_filenames:
    url,filename = url_prefix + f.replace("../../../", ""), filepath_prefix + f.replace("../../../", "")
    response = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(response.content)

# 关闭浏览器
driver.quit()


InvalidCookieDomainException: Message: invalid cookie domain
  (Session info: chrome=111.0.5563.111)
Stacktrace:
Backtrace:
	(No symbol) [0x0022DCE3]
	(No symbol) [0x001C39D1]
	(No symbol) [0x000D4C70]
	(No symbol) [0x0013A6E1]
	(No symbol) [0x0011AECC]
	(No symbol) [0x0012D57C]
	(No symbol) [0x0011ACC6]
	(No symbol) [0x000F6F68]
	(No symbol) [0x000F80CD]
	GetHandleVerifier [0x004A3832+2506274]
	GetHandleVerifier [0x004D9794+2727300]
	GetHandleVerifier [0x004DE36C+2746716]
	GetHandleVerifier [0x002D6690+617600]
	(No symbol) [0x001CC712]
	(No symbol) [0x001D1FF8]
	(No symbol) [0x001D20DB]
	(No symbol) [0x001DC63B]
	BaseThreadInitThunk [0x75D4FEF9+25]
	RtlGetAppContainerNamedObjectPath [0x77DE7BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77DE7B8E+238]
	(No symbol) [0x00000000]


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
s

# 初始化浏览器
driver = webdriver.Chrome()

# 打开登录页面
# driver.get("https://lingua.com/login/google/")



# 等待页面加载完成
# wait = WebDriverWait(driver, 10)
# wait.until(EC.presence_of_element_located((By.NAME, "identifier")))

# # 输入 Google 账号和密码
# driver.find_element(By.NAME, "identifier").send_keys("your_account")
# driver.find_element(By.NAME, "identifier").send_keys(Keys.RETURN)
# wait.until(EC.presence_of_element_located((By.NAME, "password")))
# driver.find_element(By.NAME, "password").send_keys("your_password")
# driver.find_element(By.NAME, "password").send_keys(Keys.RETURN)

# # 获取cookie
# cookies = driver.get_cookies()

# # 等待登录成功，通过检查导航栏中的用户名来判断
# wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".nav-user-avatar")))

# 将Cookie字符串解析成字典
cookie_str = 'lingua_version=EN%3B1679931692%3Breading; expires=Wed, 26-Apr-2023 15:41:32 GMT; Max-Age=2592000; path=/; domain=lingua.com; secure; httponly'
cookie_parts = cookie_str.split(';')
cookie_dict = {}
for part in cookie_parts[:-2]:
    key, value = part.strip().split('=', 1)
    cookie_dict[key] = value

# 将Cookie添加到Selenium的Cookie中
driver.add_cookie({
    'name': 'lingua_version',
    'value': cookie_dict.get('lingua_version',None),
    'expires': cookie_dict.get('expires', None),
    'Max-Age': cookie_dict.get('Max-Age', None),
    'path': cookie_dict.get('path', None),
    'domain': 'lingua.com',#'.'+cookie_dict.get('domain', None),
    'secure': cookie_dict.get('secure', None),
    'httpOnly': cookie_dict.get('httpOnly', None)
})

for cookie_name, cookie_value in cookie_dict.items():
    driver.add_cookie({'name': cookie_name, 'value': cookie_value})

# 跳转到阅读页面
driver.get("https://lingua.com/english/reading/london/")

# 等待元素出现
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "wrapper-lingua-speakers")))

# 获取页面源码
html = driver.page_source

# 解析页面源码
soup = BeautifulSoup(html, "html.parser")

# 获取所有的 data-filename 属性
for tag in soup.select(".wrapper-lingua-speakers img"):
    print(tag.get("data-filename"))
    
# 关闭浏览器
driver.quit()


InvalidCookieDomainException: Message: invalid cookie domain
  (Session info: chrome=111.0.5563.111)
Stacktrace:
Backtrace:
	(No symbol) [0x0022DCE3]
	(No symbol) [0x001C39D1]
	(No symbol) [0x000D4C70]
	(No symbol) [0x0013A6E1]
	(No symbol) [0x0011AECC]
	(No symbol) [0x0012D57C]
	(No symbol) [0x0011ACC6]
	(No symbol) [0x000F6F68]
	(No symbol) [0x000F80CD]
	GetHandleVerifier [0x004A3832+2506274]
	GetHandleVerifier [0x004D9794+2727300]
	GetHandleVerifier [0x004DE36C+2746716]
	GetHandleVerifier [0x002D6690+617600]
	(No symbol) [0x001CC712]
	(No symbol) [0x001D1FF8]
	(No symbol) [0x001D20DB]
	(No symbol) [0x001DC63B]
	BaseThreadInitThunk [0x75D4FEF9+25]
	RtlGetAppContainerNamedObjectPath [0x77DE7BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77DE7B8E+238]
	(No symbol) [0x00000000]


cookie加不進去，直接讓selenium使用我的profile

https://stackoverflow.com/questions/52394408/how-to-use-chrome-profile-in-selenium-webdriver-python-3

要下載chrome driver

https://medium.com/@bob800530/selenium-1-%E9%96%8B%E5%95%9Fchrome%E7%80%8F%E8%A6%BD%E5%99%A8-21448980dff9

要建立新的chrome profile，下載chrome portable
https://superuser.com/questions/1212958/how-to-create-a-separate-profile-in-chrome-with-same-email

In [9]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import re 

article = "London"
accent = "American"

options = webdriver.ChromeOptions()
options.add_argument("user-data-dir=D:\\ChromePortable64\\Data\\profile")
options.add_argument('profile-directory=Profile 1')
driver = webdriver.Chrome(executable_path=r'"D:\chromedriver_win32\chromedriver.exe"', chrome_options=options)


# driver.get("https://www.google.com.tw")

# 打开登录页面
driver.get("https://lingua.com/login/google/")

# 跳转到阅读页面
driver.get(f"https://lingua.com/english/reading/{article}/")

# # 等待元素出现
# wait.until(EC.presence_of_element_located((By.CLASS_NAME, "wrapper-lingua-speakers")))

# 等待元素出现
wait = WebDriverWait(driver, 10)
element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "wrapper-lingua-speakers")))

# 获取解析后的HTML内容
html = driver.execute_script("return document.documentElement.outerHTML")

# # 获取页面源码
# html = driver.page_source

# 解析页面源码
soup = BeautifulSoup(html, "html.parser")

# 获取所有的 data-filename 属性
# data_filenames = [elem['data-filename'] for elem in soup.select('[data-filename]')]
# 找到 class 属性为 "speaker-box"，并且包含 "American English" 文本的 div 元素
american_div = soup.select_one(f'.speaker-box:contains("{accent} English")')
# 获取其 data-filename 属性
data_filenames = american_div['data-filename']
url_prefix = "https://lingua.com/"
filepath_prefix = "D:/lingua/"

if type(data_filenames) == list:
    for f in data_filenames:
        url,filename = url_prefix + f.replace("../../../", ""), filepath_prefix + f.replace("../../../", "")
        filename = filename.replace(re.search(r'english-\w+-(.*).mp3', filename).group(1),accent)
        response = requests.get(url)
        with open(filename, 'wb') as f:
            f.write(response.content)
else:
    url,filename = url_prefix + data_filenames.replace("../../../", ""), filepath_prefix + data_filenames.replace("../../../", "")
    filename = filename.replace(re.search(r'english-\w+-(.*).mp3', filename).group(1),accent)
    response = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(response.content)
        
# 关闭浏览器
driver.quit()


C:\Users\max\AppData\Local\Temp\ipykernel_6924\2322894684.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r'"D:\chromedriver_win32\chromedriver.exe"', chrome_options=options)
C:\Users\max\AppData\Local\Temp\ipykernel_6924\2322894684.py:16: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=r'"D:\chromedriver_win32\chromedriver.exe"', chrome_options=options)
